In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from transformers import AutoTokenizer, AutoModel
import os
from dotenv import load_dotenv

print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
hf_token = os.getenv("HF_API_KEY")  

tokenizer = AutoTokenizer.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0", 
    token=hf_token
)
model = AutoModel.from_pretrained(
    "Cohere/Cohere-embed-multilingual-v3.0",
    token=hf_token,
    torch_dtype="auto"
)
# Örnek metinler
texts = ["Bu bir test cümlesidir.", "Mahkeme kararını verdi."]

# Tokenize et
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Modeli kullanarak embedding al
with torch.no_grad():
    outputs = model(**inputs)
    # Burada last_hidden_state'den mean pooling yapıyoruz
    embeddings = outputs.last_hidden_state.mean(dim=1)

print("Embedding boyutu:", embeddings.shape)
print("İlk embedding örneği:", embeddings[0])


In [ ]:
!echo $HF_API_KEY

In [ ]:
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv
print(load_dotenv("/home/yapayzeka/ahsen_bulbul/qdrant/.env"))
# Hugging Face API key ortam değişkeninden çek
hf_token = os.getenv("HF_API_KEY")

# Modeli token ile yükle
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=hf_token  # API key burada
)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

embeddings = model.encode(sentences)

# Benzerlik hesapla
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
